In [ ]:
!pip install google-cloud-documentai
!pip install google-cloud-storage

In [3]:
from google.cloud import documentai
from google.cloud import storage
import os
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

# Configure Document AI client
project_id = "sodium-coil-470706-f4"  
location = "us"  # Change if using different region
processor_id = "18d898182b219656"

# Initialize the client
client_options = {"api_endpoint": f"{location}-documentai.googleapis.com"}
client = documentai.DocumentProcessorServiceClient(client_options=client_options)

c:\Users\user\OneDrive\Documents\GenAI-exchange\backend\venv\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [17]:
def process_document(file_path):
    """Process a document using Document AI. Accepts local file path or gs:// URI."""
    name = f"projects/{project_id}/locations/{location}/processors/{processor_id}"

    # Use GCS input config for Document AI
    gcs_document = documentai.GcsDocument(
        gcs_uri=file_path,
        mime_type="application/pdf"  # Update mime type if needed
    )

    request = documentai.ProcessRequest(
        name=name,
        gcs_document=gcs_document
    )
    result = client.process_document(request=request)
    return result.document

In [7]:
def display_results(document):
    """Display the extracted text and other information"""
    print("Full text extracted:")
    print("="*50)
    print(document.text)
    
    print("\nForm fields found:")
    print("="*50)
    for page in document.pages:
        for field in page.form_fields:
            name = get_text(field.field_name, document)
            value = get_text(field.field_value, document)
            print(f"{name}: {value}")
            
def get_text(doc_element, document):
    """Extract text from a document element"""
    text = ""
    for segment in doc_element.text_anchor.text_segments:
        start_index = segment.start_index
        end_index = segment.end_index
        text += document.text[start_index:end_index]
    return text

In [18]:
# Test the implementation
test_file_path = r"gs://legal-doc-bucket1/generative-ai_pdf_certificate_of_incoporation.pdf"  # Update with your test file path

try:
    # Process the document
    processed_doc = process_document(test_file_path)
    
    # Display results
    display_results(processed_doc)
    # Print confidence score
    print("\nDocument confidence score:", processed_doc.text_quality_scores[0].score)
    
except Exception as e:
    print(f"An error occurred: {str(e)}")

Full text extracted:
7636
June 19, 2023
State of Rhode Island
Department of State | Business Services Division
Gregg M. Amore, Secretary of State
Dear Sir or Madam:
The Division of Business Services of the Office of the Secretary of State has not received your Annual
Report for the year 2023 that was due between February 1 and May 1, 2023.
Pursuant to the provisions set forth in R.I.G.L. 7-1.2-1310 and 7-1.2-1414, the Certificate of
Incorporation/Authority of the above-named entity will be revoked after 60 days from the date of this
notice for failure to file the Annual Report.
R.I.G.L. 7-1.2-1501(e) states that any corporation failing or refusing to file its annual report for any year
within thirty (30) days after the time prescribed by law is subject to a penalty fee of $25.00. Due to the late
filing, the fee for the 2023 annual report is now $75.00.
The easiest way to complete this filing is to file online at https://www.sos.ri.gov/divisions/business-
services and pay by credit card